In [1]:
from abc import ABCMeta, abstractmethod
import random
import math
import pandas as pd
import numpy as np

In [2]:
class Datos:

    # Constructor: procesar el fichero para asignar correctamente las
    # variables nominalAtributos, datos y diccionario
    def __init__(self, nombreFichero):

        # Leemos el dataset pasado por argumentos, y lo canvertimos en un DataFrame
        # de pandas
        df = pd.read_csv(nombreFichero)
        # Listas de valores nominales (True), o numericos (False)
        self.nominalAtributos = []
        # conversor de datos nominales a datos numéricos
        self.diccionarios = {}
        # Array numpy de datos convertidos a forma numerica
        self.datos = np.array([])

        # Iterar por los atributos (incluye clase)
        for attr in df.columns:
            # Crear una entrada al diccionario para cada atributo
            self.diccionarios[attr] = {}
            # Se guarda si es atributo nominal o no, en el array nominalAtributos
            if df[attr].dtypes in [np.int64, np.float64]:
                self.nominalAtributos.append(False)
            elif isinstance(df[attr].dtypes, object):
                self.nominalAtributos.append(True)
                value = 1
                # Se iteran los datos únicos obtenidos para cada atributo nominal, de manera ordenanda
                for key in sorted(df[attr].unique()):
                    # Se crea una nueva entrada para cada dato nominal con el valor al cual se va a tranformar
                    self.diccionarios[attr][key] = value
                    value += 1
            else:
                raise ValueError

            # Con el metodo replace, reemplazara los datos de nuestro DataFrame con el mapeo de claves-valor realizadas en diccionarios
            df = df.replace(self.diccionarios)

            # Finalmente se pasa a array de numpy
            self.datos = df.to_numpy()

    def extraeDatos(self, idx):

        # Lista que almacenará los datos correspondientes a los indices pedidos
        listaDatos = []

        for i in idx:
            listaDatos.append(self.datos[i])

        return listaDatos

In [19]:
class Particion():

    # Esta clase mantiene la lista de �ndices de Train y Test para cada partici�n del conjunto de particiones
    def __init__(self):
        self.indicesTrain = []
        self.indicesTest = []

class EstrategiaParticionado:

    # Clase abstracta
    __metaclass__ = ABCMeta

    # Atributos: deben rellenarse adecuadamente para cada estrategia concreta. Se pasan en el constructor
    def __init__(self):
        self.particiones = []

    @abstractmethod
    # TODO: esta funcion deben ser implementadas en cada estrategia concreta
    def creaParticiones(self, datos, seed=None):
        pass

class ValidacionSimple(EstrategiaParticionado):

    def __init__(self, numeroEjec, propTest):
        self.numeroEjecuciones = numeroEjec
        self.proporcionTest = propTest
        super().__init__()

    # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado y el n�mero de ejecuciones deseado
    # Devuelve una lista de particiones (clase Particion)
    # TODO: implementar
    def creaParticiones(self, datos, seed=None):
        # Se obtiene el numero de filas y se crea una lista con tantos indices como elementos haya en la lista
        filas = len(datos.datos)
        indices = [i for i in range(0, filas)]
        # Se iteran entre el numero de ejecuciones que se quieren realizar
        for i in range(0, self.numeroEjecuciones):
            # Se hace un mezclado de las filas (indices) del dataset
            random.shuffle(indices)
            # Se crea un nuevo objeto de particionado
            part = Particion()
            # Se añaden los indices de test e indices de entrenamiento en funcion de la proprcion de Test
            part.indicesTrain.extend(indices[0: self.proporcionTest - 1])
            part.indicesTest.extend(indices[self.proporcionTest: filas - 1])
            # Se añade a la lista de particiones, la nueva particion creada
            self.particiones.append(part)

class ValidacionCruzada(EstrategiaParticionado):

    def __init__(self, numParticiones):
        self.numeroParticiones = numParticiones
        super().__init__()

    # Crea particiones segun el metodo de validacion cruzada.
    # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
    # Esta funcion devuelve una lista de particiones (clase Particion)
    # TODO: implementar

    def creaParticiones(self, datos, seed=None):
        # Se obtiene el numero de filas y se calcula cuanto es el tamaño de particion, dependiendo de cuantas queremos
        filas = len(datos.datos)
        indices = [i for i in range(0, filas)]
        tamParticion = math.ceil(filas / self.numeroParticiones)

        # Se itera entre el numero de particiones
        for i in range(0, self.numeroParticiones):

            # Se crea una nueva particione
            part = Particion()
            # La parte de test la sacamos desde el indice que estamos hasta el indice + 1
            # por el tamaño de la particion
            part.indicesTest.extend(indices[tamParticion *
                                            i: tamParticion * (i + 1)])
            # La parte de train,se divide en la parte anterior y posterior del train
            part.indicesTrain.extend(indices[0: tamParticion * i])
            part.indicesTrain.extend(indices[tamParticion * (i + 1):])

            # Se añade a la lista de particiones
            self.particiones.append(part)


In [21]:
# MOSTRAREMOS SOLO 5 PARTICIONES POR CADA ESTRATEGIA PARA QUE SE PUEDAN APRECIAR
# MEJOR LOS RESULTADOS

datos = Datos("tic-tac-toe.csv")
print("Contenido del diccionario:\n" + str(datos.diccionarios))
print("Contenido matriz datos:\n" + str(datos.datos[0]) + "\n" + str(datos.datos[1]))

estrategia = ValidacionCruzada(50)
estrategia.creaParticiones(datos)
print("-----------------------------------------------------------------------------------------------------------------------------------")
print("Numero de particiones: " + str(estrategia.numeroParticiones))
print("Indices Validacion Cruzada:")
for i in range(0, 5):
    print("Indices " + str(i) + " Train: " +
          str(estrategia.particiones[i].indicesTrain))
    print("Indices " + str(i) + " Test: " +
          str(estrategia.particiones[i].indicesTest))

Contenido del diccionario:
{'TLeftSq': {'b': 1, 'o': 2, 'x': 3}, 'TMidSq': {'b': 1, 'o': 2, 'x': 3}, 'TRightSq': {'b': 1, 'o': 2, 'x': 3}, 'MLeftSq': {'b': 1, 'o': 2, 'x': 3}, 'MMidSq': {'b': 1, 'o': 2, 'x': 3}, 'MRightSq': {'b': 1, 'o': 2, 'x': 3}, 'BLeftSq': {'b': 1, 'o': 2, 'x': 3}, 'BMidSq': {'b': 1, 'o': 2, 'x': 3}, 'BRightSq': {'b': 1, 'o': 2, 'x': 3}, 'Class': {'negative': 1, 'positive': 2}}
Contenido matriz datos:
[3 3 3 3 2 2 3 2 2 2]
[3 3 3 3 2 2 2 3 2 2]
-----------------------------------------------------------------------------------------------------------------------------------
Numero de particiones: 50
Indices Validacion Cruzada
Indices 0 Train: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 10